In [2]:
import pandas as pd
import uuid

class Question:
  questionId: str
  questionText: str
  answerText: str
  def __init__(self, qId: str, qText: str,aText: str) -> None:
        self.questionId = qId
        self.questionText = qText
        self.answerText = aText
  

questions=[]
df = pd.read_csv("alzheimer_questionaire.csv",encoding = 'cp1252')
print(df.head())
#df=df.sample(5)


                                            question  \
0            Did you feel that you forget something?   
1        If so, is it happening more than one year?    
2          Did you repeat some works again and again   
3  How you manage your appointments? Did you keep...   
4  How do you keep your house chores? Do you arra...   

                              answer  Result  
0    Sometimes I forget some matters       1  
1  It is happening for several years       1  
2        I am very good at gardening       0  
3              I go to park everyday       0  
4    I always keep in my arrangement       1  


In [3]:
from torch.nn.functional import softmax
from transformers import BertForNextSentencePrediction, BertTokenizer

C:\Users\dines\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# load pretrained model and a pretrained tokenizer
model = BertForNextSentencePrediction.from_pretrained('bert-base-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
df.shape

(49, 3)

In [7]:
matchCount=0
for i, row in df.iterrows():
  encoded = tokenizer.encode_plus(row['question'], text_pair=row['answer'], return_tensors='pt')
  #print(encoded)
  seq_relationship_logits = model(**encoded)[0]
  probs = softmax(seq_relationship_logits, dim=1)
  val= probs[0][0].item()
  if(val>.7):
    val=1
  else:
    val=0
  if(val==row['Result']):
    matchCount=matchCount+1
accuracy= (matchCount/i)*100

print('accuracy%:',accuracy)
    

accuracy%: 72.91666666666666
